# DLC/ANIPOSE PIPELINE

Automatically converts fly videos to estimated 3D coordinates using DeepLabCut and Anipose

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)
logging.debug("Logging works :)")

In [ ]:
# Next two lines are for dev purposes only 
%load_ext autoreload
%autoreload 2
from pathlib import Path
from dlc import analyze_new
from preprocess import gen_anipose_files, clean_dfs 
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### STEP 0: Configuration

- Add filepath to DLC network config files (e.g `dlc_networks.yml`)
- Add filepath to folder with experiment videos

In [9]:
# Set the filepath to the geontype folder containing videos
videos = r""
videos = Path(videos)


p_networks = Path(r'./common_files/dlc_networks.yml')

### STEP 1: Running DeepLabCut

This will run on a directory with fly video files and generate DLC pose estimations, outputting to the same directory.

##### Processing done:
- DeepLabCut `analyze_videos`
- DeepLabCut `filterpredictions`

In [ ]:
analyze_new(videos, p_networks)
print("Finished DLC analysis...")

### STEP 2: Preprocessing for anipose

##### Processing done:
- Fix points
- Remove columns
- Rename (GenotypeFly#-CamName)
- Convert to .h5
- Generate anipose file structure

In [ ]:
error = False

# find all the CSVs that DLC generated
processed_dirs = {}
for p_csv in utils.get_csvs(videos): # get all CSVs
    parent_dir = p_csv.parent.parent.parent

    # Pick current generated, filtered CSV 
    
    # TODO: also check for cam name and model name
    csv_name = p_csv.stem
    if "filtered" not in csv_name:
        # only run on filtered CSVs
        continue

    # Fix points, remove columns 
    csv_df = clean_dfs(p_csv)

    processed_csv = (csv_df, p_csv)
    if parent_dir in processed_dirs:
        processed_dirs[parent_dir].append(processed_csv) # Append to list of processed CSVs under that parent directory
    else:
        processed_dirs[parent_dir] = [processed_csv] # Create list of processed CSVs under that parent directory

# Generate anipose file structure
p_calibration_target = Path(r"./common_files/calibration_target.yml") # calibration target config file 
p_calibration_timeline = Path(r"./common_files/calib_timeline.yml") # calibration timeline config file
p_gcam_dummy = Path(r"./common_files/GenotypeFly-G.h5") # Dummy file for G camera (top-down view)

# check that all the files exist
if not p_calibration_target.exists():
    logging.error("`calibration_target.yml` does not exist.")
    error = True
if not p_calibration_timeline.exists():
    logging.error("`calib_timeline.yml` does not exist.")
    error = True
if not p_gcam_dummy.exists():
    logging.error("`GenotypeFly-G.h5` (G camera dummy file) does not exist.")
    error = True

if not error:
    logging.info("Generating anipose files...")
    for parent_dir, processed_csvs in processed_dirs:
        if not gen_anipose_files(parent_dir, p_networks, p_calibration_target, p_calibration_timeline, processed_csvs, p_gcam_dummy):
            # TODO: gen_anipose_files needs to return somethng when it finishes (maybe directory where it was generated)
            print(f"[WARNING] Skipped anipose generation for {parent_dir}")
    print('Finished preprocessing...')
else:
    print("Terminated due to error.")